# function #

In [1]:
import os
import logging
import sys
import random
import numpy as np
import pandas as pd
import h5py
import tables
import argparse as ap

import scipy
import scipy.sparse as sp_sparse
import scipy.stats as ss

import collections
from collections import defaultdict

import gensim
from gensim.models import LdaModel
from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score

from ModelEvaluate import *
from ModelTrain import *
from Deconvolution import *
#import stride

In [2]:
def setup_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    # torch.backends.cudnn.deterministic = True

In [3]:
import scanpy as sc
import anndata

def MarkerFind(sc_count_mat, sc_count_genes, sc_count_cells, sc_anno_file, ntop = 200):
    '''
    Find markers for each celltype
    '''
    adata = anndata.AnnData(sc_count_mat.transpose(), obs = dict(obs_names = sc_count_cells), var = dict(var_names = sc_count_genes))
    # preprocess
    sc.pp.filter_cells(adata, min_genes = 200)
    sc.pp.filter_genes(adata, min_cells = 10)
    sc.pp.calculate_qc_metrics(adata, inplace = True)
    # normalize data
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, n_top_genes = 2000)
    # add celltype info
    meta_data = pd.read_csv(sc_anno_file, sep = "\t", header = None, index_col = 0)
    meta_data.columns = ["Celltype"]
    adata.obs["Celltype"] = meta_data.loc[adata.obs.index, "Celltype"]
    # find marker genes for each cell-type
    sc.tl.rank_genes_groups(adata, groupby = 'Celltype', method='wilcoxon', pts = True, use_raw = False, tie_correct = True)
    top_marker_df = pd.DataFrame(adata.uns['rank_genes_groups']['names']).iloc[0:ntop,]
    top_marker_array = np.array(top_marker_df.T)
    top_marker_list = list(set(top_marker_array.flatten().tolist()))
    return(top_marker_list)

In [4]:
FeatureBCMatrix = collections.namedtuple('FeatureBCMatrix', ['ids', 'names', 'barcodes', 'matrix'])
def read_10X_h5(filename):
    """Read 10X HDF5 files, support both gene expression and peaks."""
    with tables.open_file(filename, 'r') as f:
        try:
            group = f.get_node(f.root, 'matrix')
        except tables.NoSuchNodeError:
            print("Matrix group does not exist in this file.")
            return None
        feature_group = getattr(group, 'features')
        ids = getattr(feature_group, 'id').read()
        names = getattr(feature_group, 'name').read()
        barcodes = getattr(group, 'barcodes').read()
        data = getattr(group, 'data').read()
        indices = getattr(group, 'indices').read()
        indptr = getattr(group, 'indptr').read()
        shape = getattr(group, 'shape').read()
        matrix = sp_sparse.csc_matrix((data, indices, indptr), shape=shape)
        return FeatureBCMatrix(ids, names, barcodes, matrix)

def write_10X_h5(filename, matrix, features, barcodes):
    """Write 10X HDF5 files, support both gene expression and peaks."""
    f = h5py.File(filename, 'w')
    datatype = "Gene"
    M = sp_sparse.csc_matrix(matrix, dtype=np.float32)
    B = np.array(barcodes, dtype='|S200')
    P = np.array(features, dtype='|S100')
    FT = np.array([datatype]*len(features), dtype='|S100')
    mat = f.create_group('matrix')
    mat.create_dataset('barcodes', data=B)
    mat.create_dataset('data', data=M.data)
    mat.create_dataset('indices', data=M.indices)
    mat.create_dataset('indptr', data=M.indptr)
    mat.create_dataset('shape', data=M.shape)
    fet = mat.create_group('features')
    fet.create_dataset('id', data=P)
    fet.create_dataset('name', data=P)
    f.close()
    
def read_count(count_file, separator = "tab"):
    """Read count table as matrix."""

    if separator == "tab":
        sep = "\t"
    elif separator == "space":
        sep = " "
    elif separator == "comma":
        sep = ","
    else:
        raise Exception("Invalid separator!")

    infile = open(count_file, 'r').readlines()
    barcodes = infile[0].strip().split(sep)
    features = []
    matrix = []
    for line in infile[1:]:
        line = line.strip().split(sep)
        features.append(line[0])
        matrix.append([float(t) for t in line[1:]])
    if len(barcodes) == len(matrix[0]) + 1:
        barcodes = barcodes[1:]

    return {"matrix": matrix, "features": features, "barcodes": barcodes}

In [5]:
def stProcess(st_count_file, st_scale_factor = None):
    # read spatial count file
    if st_count_file.endswith(".h5"):    
        st_count = read_10X_h5(st_count_file)
        st_count_mat = st_count.matrix
        st_count_genes = st_count.names.tolist()
        st_count_spots = st_count.barcodes.tolist()
        if type(st_count_genes[0]) == bytes:
            st_count_genes = [i.decode() for i in st_count_genes]
        if type(st_count_spots[0]) == bytes:
            st_count_spots = [i.decode() for i in st_count_spots]
    else:
        st_count = read_count(st_count_file)
        st_count_mat = st_count["matrix"]
        st_count_mat = sp_sparse.csc_matrix(st_count_mat, dtype=np.float32)
        st_count_genes = st_count["features"]
        st_count_spots = st_count["barcodes"]
    # scale the count matrix
    count_per_spot = np.asarray(st_count_mat.sum(axis=0))
    count_per_spot = np.array(count_per_spot.tolist()[0])
    if not st_scale_factor:
        st_scale_factor = np.round(np.quantile(count_per_spot, 0.75)/1000, 0)*1000
    r,c = st_count_mat.nonzero()
    count_per_spot_sp = sp_sparse.csr_matrix(((1.0/count_per_spot)[c], (r,c)), shape=(st_count_mat.shape))
    st_count_scale_mat = st_count_mat.multiply(count_per_spot_sp)*st_scale_factor
    st_count_scale_mat = sp_sparse.csc_matrix(st_count_scale_mat)

    return({'scale_matrix': st_count_scale_mat, "raw_matrix": st_count_mat,"genes": st_count_genes, "spots": st_count_spots})

def scProcess(sc_count_file, sc_anno_file, out_dir, out_prefix, sc_scale_factor = None):
    # read scRNA-seq data
    if sc_count_file.endswith(".h5"):    
        sc_count = read_10X_h5(sc_count_file)
        sc_count_mat = sc_count.matrix
        sc_count_genes = sc_count.names.tolist()
        sc_count_cells = sc_count.barcodes.tolist()
        if type(sc_count_genes[0]) == bytes:
            sc_count_genes = [i.decode() for i in sc_count_genes]
        if type(sc_count_cells[0]) == bytes:
            sc_count_cells = [i.decode() for i in sc_count_cells]
        h5_filename = sc_count_file
    else:
        sc_count = read_count(sc_count_file)
        sc_count_mat = sc_count["matrix"]
        sc_count_mat = sp_sparse.csc_matrix(sc_count_mat, dtype=np.float32)
        sc_count_genes = sc_count["features"]
        sc_count_cells = sc_count["barcodes"]
        h5_filename = os.path.join(out_dir, "%s_scRNA_count.h5" %(out_prefix))
        write_10X_h5(filename = h5_filename, matrix = sc_count_mat,
                     features = sc_count_genes, barcodes = sc_count_cells)

    # scale the count matrix
    count_per_cell = np.asarray(sc_count_mat.sum(axis=0))
    count_per_cell = np.array(count_per_cell.tolist()[0])
    if not sc_scale_factor:
        sc_scale_factor = np.round(np.quantile(count_per_cell, 0.75)/1000, 0)*1000
    r,c = sc_count_mat.nonzero()
    count_per_cell_sp = sp_sparse.csr_matrix(((1.0/count_per_cell)[c], (r,c)), shape=(sc_count_mat.shape))
    sc_count_scale_mat = sc_count_mat.multiply(count_per_cell_sp)*sc_scale_factor
    sc_count_scale_mat = sp_sparse.csc_matrix(sc_count_scale_mat)
    # read cell-type meta file
    cell_celltype_dict = {}
    for line in open(sc_anno_file, "r"):
        items = line.strip().split("\t")
        cell_celltype_dict[items[0]] = items[1]

    return({'scale_matrix': sc_count_scale_mat, "raw_matrix": sc_count_mat,
        "genes": sc_count_genes, "cells": sc_count_cells, "cell_celltype": cell_celltype_dict})


In [6]:
def LDA(sc_corpus, ntopics, genes_dict, genes_shared, cell_gene_list, sc_count_cells, cell_celltype_list, model_dir):
    lda = LdaModel(corpus = sc_corpus, num_topics = ntopics, id2word = genes_dict)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_file = os.path.join(model_dir, "lda_model_%s" %(ntopics))
    lda.save(model_file)
    # compute the coherence
    cm = CoherenceModel(model = lda, corpus = sc_corpus, coherence='u_mass')
    umass_coherence = cm.get_coherence()
    cm = CoherenceModel(model = lda, corpus = sc_corpus, texts = cell_gene_list, coherence='c_v')
    cv_coherence = cm.get_coherence()
    # save the topic-cell matrix
    topic_cell = lda.get_document_topics(sc_corpus)
    topic_cell_mat = gensim.matutils.corpus2csc(topic_cell)
    topic_cell_file = os.path.join(model_dir, "topic_cell_mat_%s.npz" %(ntopics))
    topic_cell_df_file = os.path.join(model_dir, "topic_cell_mat_%s.txt" %(ntopics))
    scipy.sparse.save_npz(topic_cell_file, topic_cell_mat)
    topic_cell_df = pd.DataFrame(topic_cell_mat.todense(), 
        index = ["Topic %s" %i for i in range(1, 1 + topic_cell_mat.shape[0])], 
        columns = sc_count_cells)
    topic_cell_df.to_csv(topic_cell_df_file, sep = "\t", index = True, header = True)
    # save the gene-topic matrix
    topic_gene_mat_list = lda.get_topics()
    topic_gene_mat = np.array(topic_gene_mat_list)
    gene_topic_mat = topic_gene_mat.transpose()
    gene_topic_mat_list = gene_topic_mat.tolist()
    gene_topic_file = os.path.join(model_dir, "gene_topic_mat_%s.txt" %(ntopics))
    gene_topic_out = open(gene_topic_file, "w")
    gene_topic_out.write("\t".join(["Topic%s" %i for i in range(1, ntopics + 1)]) + "\n")
    for i in range(len(gene_topic_mat_list)):
        gene_topic_out.write(genes_shared[i] + "\t" + "\t".join([str(j) for j in gene_topic_mat_list[i]]) + "\n")
    gene_topic_out.close()
    # convert topic_cell_mat to topic_celltype_mat
    celltype_topic_dict = {}
    celltype_num_dict = {}
    celltypes = sorted(list(set(cell_celltype_list)))
    for celltype in celltypes:
        celltype_topic_dict[celltype] = [0]*ntopics
        celltype_num_dict[celltype] = 0
    for i in range(topic_cell_mat.shape[1]):
        cell_celltype = cell_celltype_list[i]
        celltype_topic_dict[cell_celltype] = [celltype_topic_dict[cell_celltype][j] + topic_cell_mat[j,i] for j in range(topic_cell_mat.shape[0])]
        celltype_num_dict[cell_celltype] = celltype_num_dict[cell_celltype] + 1
    celltype_topic_mean_dict = {}
    for celltype in celltypes:
        celltype_topic_mean_dict[celltype] = [i/celltype_num_dict[celltype] for i in celltype_topic_dict[celltype]]
    topic_celltype_df = pd.DataFrame(data = celltype_topic_mean_dict)
    topic_celltype_file = os.path.join(model_dir,"topic_celltype_mat_%s.txt" %(ntopics))
    topic_celltype_df.to_csv(topic_celltype_file, sep="\t")
    # return results
    res_dict = {"coherence": [umass_coherence, cv_coherence], 
    "topic_cell_mat": topic_cell_mat, 
    "topic_celltype_df": topic_celltype_df,
    "celltype_num_dict": celltype_num_dict}

    return(res_dict)

def scLDA(sc_count_mat, sc_count_genes, sc_count_cells, cell_celltype_dict,
          st_count_mat, st_count_genes, st_count_spots,
          normalize, gene_use, ntopics_list, out_dir):
    sc_count_genes_array = np.array(sc_count_genes)
    sc_count_genes_sorter = np.argsort(sc_count_genes_array)
    if normalize:
        sc_count_mat = StandardScaler(with_mean=False).fit_transform(sc_count_mat.transpose()).transpose()
    if gene_use == "All":
        genes_shared = list(set(st_count_genes) & set(sc_count_genes))
    else:
        genes_shared = list(set(st_count_genes) & set(sc_count_genes) & set(gene_use))
    genes_shared = sorted(genes_shared)
    genes_shared_array = np.array(genes_shared)
    genes_shared_index = sc_count_genes_sorter[np.searchsorted(sc_count_genes_array, genes_shared_array, sorter = sc_count_genes_sorter)]
    sc_count_mat_use = sc_count_mat[genes_shared_index,:]
    cell_gene_list = []
    sc_count_mat_use_nonzero = sc_count_mat_use.nonzero()
    for i in range(sc_count_mat_use.shape[1]):
        gene_ind = sc_count_mat_use_nonzero[0][sc_count_mat_use_nonzero[1] == i]
        genes = genes_shared_array[gene_ind].tolist()
        cell_gene_list.append(genes)
        # evaluate the model
    # construct single-cell gene corpus
    sc_corpus = gensim.matutils.Sparse2Corpus(sc_count_mat_use)
    genes_dict = Dictionary([genes_shared])
    genes_dict_file = os.path.join(out_dir, "Gene_dict.txt")
    genes_dict.save_as_text(genes_dict_file)
    cell_celltype_list = []
    for i in range(len(sc_count_cells)):
        cell_celltype = cell_celltype_dict[sc_count_cells[i]]
        cell_celltype_list.append(cell_celltype)
    print("Selecting the optimal model.")
    model_selection_res = ModelSelect(sc_corpus = sc_corpus, genes_dict = genes_dict, genes_shared = genes_shared,
        ntopics_list = ntopics_list, cell_gene_list = cell_gene_list, sc_count_cells = sc_count_cells, 
        cell_celltype_list = cell_celltype_list, out_dir = out_dir)
    model_selected = model_selection_res["model"]
    ntopics_selected = model_selection_res["ntopics"]

    return({"genes_dict": genes_dict, "model_selected": model_selected, "ntopics_selected": ntopics_selected})

In [7]:
def JS_divergence(p,q):
    M = (p+q)/2
    return 0.5 * ss.entropy(p,M,base=2) + 0.5*ss.entropy(q,M,base=2)

# 矩阵按行求JSD,
def JS_divergence_mat(p_mat, q_mat):
    raw = p_mat.shape[0]
    jsd = 0
    for i in range(raw):
        jsd += JS_divergence(p_mat.iloc[i], q_mat.iloc[i])
    return jsd

# RUN #

In [67]:
#data_dir = '/data/lyx/hubs/SpaTD/stdgcn/benchmark_data/MERFISH/'
# idx1 = 1
# idx2 = 0.01
# res = 50
seed=80
setup_seed(seed)
data_dir = '/data/lyx/hubs/SpaTD/stdgcn/benchmark_data/seqFISH_plus/Dataset2_seqFISHplus_AllenVIsp'
prefix = "seqFISHplus"
res = 200
idx1 = 'all'
os.chdir('/data/lyx/hubs/SpaTD/stdgcn/benchmark/seqFISH_plus/Dataset2_seqFISHplus_AllenVIsp/')

In [68]:
#sc_anno = pd.read_table(os.path.join(data_dir,"sc_data","sc_label.tsv"))
#sc_anno.to_csv(os.path.join(data_dir,"sc_data","sc_label_for_stride.tsv"),index=None,header=None,sep="\t")

In [69]:
#sc_count = pd.read_table(os.path.join(data_dir,"sc_data","MERFISH_ID{0}_{1}_sc_data.tsv".format(idx1,idx2)),index_col=0)
#sc_count.transpose().to_csv(os.path.join(data_dir,"sc_data","MERFISH_ID{0}_{1}_sc_data_tanspose.tsv".format(idx1,idx2)) ,sep="\t")

In [70]:
st_count = pd.read_table(os.path.join(data_dir,"ST_data","seqFishplus_Cortex_{0}_data_all.tsv".format(res)),index_col=0)
st_count.transpose().to_csv(os.path.join(data_dir,"ST_data","seqFishplus_Cortex_{0}_data_all_transpose.tsv".format(res)) ,sep="\t")

In [71]:
# sc_count_file = os.path.join(data_dir,"sc_data","MERFISH_ID{0}_{1}_sc_data_tanspose.tsv".format(idx1,idx2)) 
# sc_anno_file = os.path.join(data_dir,"sc_data","sc_label_for_stride.tsv")
# st_count_file = os.path.join(data_dir,"ST_data","MERFISH_ID{0}_data_{1}_tanspose.tsv".format(idx1,idx2)) 
sc_count_file = os.path.join(data_dir,"sc_data",'sc_data_transpose.tsv') 
sc_anno_file = os.path.join(data_dir,"sc_data","sc_label_for_stride.tsv")
st_count_file = os.path.join(data_dir,"ST_data","seqFishplus_Cortex_{0}_data_all_transpose.tsv".format(res)) 
model_dir = None
sc_scale_factor = None
st_scale_factor = None
out_dir = "stride"
out_prefix = "seqFISHplus_dataset2"
normalize = True
# gene_use = pd.read_table(os.path.join('/data/lyx/hubs/SpaTD/stdgcn/benchmark/seqFISH_plus/Dataset3_Cortex_allField_77spot/',
#                                       "marker_genes.tsv"),header=None)[0].to_list()
ntopics_list = None

if not os.path.exists(out_dir):
        os.makedirs(out_dir)

In [72]:
print("Reading ST matrix...")
st_info = stProcess(st_count_file, st_scale_factor)
st_count_mat = st_info["scale_matrix"]
st_count_genes = st_info["genes"]
st_count_spots = st_info["spots"]

Reading ST matrix...


In [73]:
print("Reading single-cell count matrix...")
sc_info = scProcess(sc_count_file, sc_anno_file, out_dir, out_prefix, sc_scale_factor)
sc_count_scale_mat = sc_info["scale_matrix"]
sc_count_raw_mat = sc_info["raw_matrix"]
sc_count_genes = sc_info["genes"]
sc_count_cells = sc_info["cells"]
cell_celltype_dict = sc_info["cell_celltype"]

Reading single-cell count matrix...


In [74]:
print("Identifying markers...")
findmarker = True
gene_use = MarkerFind(sc_count_raw_mat, sc_count_genes, sc_count_cells, sc_anno_file, ntop = 200)
celltypes = set(cell_celltype_dict.values())

Identifying markers...


In [75]:
ntopics_list = list(range(len(celltypes), 3*len(celltypes)+1))

In [76]:
sc_count_genes_array = np.array(sc_count_genes)
sc_count_genes_sorter = np.argsort(sc_count_genes_array)

In [77]:
sc_count_mat = StandardScaler(with_mean=False).fit_transform(sc_count_scale_mat.transpose()).transpose()

In [78]:
# genes_shared = list(set(st_count_genes) & set(sc_count_genes) & set(gene_use))
genes_shared = list(set(st_count_genes) & set(gene_use))
print(len(genes_shared))

1064


In [79]:
genes_shared = sorted(genes_shared)
genes_shared_array = np.array(genes_shared)
genes_shared_index = sc_count_genes_sorter[np.searchsorted(sc_count_genes_array, 
                                                           genes_shared_array, 
                                                           sorter = sc_count_genes_sorter)]
sc_count_mat_use = sc_count_mat[genes_shared_index,:]

In [80]:
cell_gene_list = []
sc_count_mat_use_nonzero = sc_count_mat_use.nonzero()
for i in range(sc_count_mat_use.shape[1]):
    gene_ind = sc_count_mat_use_nonzero[0][sc_count_mat_use_nonzero[1] == i]
    genes = genes_shared_array[gene_ind].tolist()
    cell_gene_list.append(genes)

In [81]:
sc_corpus = gensim.matutils.Sparse2Corpus(sc_count_mat_use)
genes_dict = Dictionary([genes_shared])
genes_dict_file = os.path.join(out_dir, "Gene_dict.txt")
genes_dict.save_as_text(genes_dict_file)

In [82]:
cell_celltype_list = []
for i in range(len(sc_count_cells)):
    cell_celltype = cell_celltype_dict[sc_count_cells[i]]
    cell_celltype_list.append(cell_celltype)
print("Selecting the optimal model.")

Selecting the optimal model.


In [83]:
print("Training topic model...")
lda_res = scLDA(sc_count_scale_mat, sc_count_genes, sc_count_cells, cell_celltype_dict,
                st_count_mat, st_count_genes, st_count_spots,
                normalize, gene_use, ntopics_list, out_dir)

genes_dict = lda_res["genes_dict"]
model_selected = lda_res["model_selected"]
ntopics_selected = lda_res["ntopics_selected"]
model_dir = os.path.join(out_dir, "model")
print("Deconvolving spatial transcriptomics...")

Training topic model...
Selecting the optimal model.
Number of topics: 6


/data/lyx/software/anaconda3/envs/stride/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Number of topics: 7


/data/lyx/software/anaconda3/envs/stride/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Number of topics: 8


...

/data/lyx/software/anaconda3/envs/stride/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Number of topics: 18


/data/lyx/software/anaconda3/envs/stride/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Deconvolving spatial transcriptomics...


<Figure size 640x480 with 0 Axes>

In [84]:
metrics_df =pd.read_csv(os.path.join(out_dir, "Model_selection.txt"), sep="\t")
model_dir = os.path.join(out_dir, "model")

In [85]:
print("Deconvolving spatial transcriptomics...")
spot_celltype_array_norm_df = SpatialDeconvolve(st_count_mat, st_count_genes, st_count_spots, 
                                                genes_dict, model_selected, ntopics_selected, 
                                                normalize, out_dir, model_dir, out_prefix)

Deconvolving spatial transcriptomics...


In [86]:
out_dir = '/data/lyx/hubs/SpaTD/stdgcn/benchmark/seqFISH_plus/Dataset2_seqFISHplus_AllenVIsp/'

In [87]:
# spot_celltype_array_norm_df.to_csv("/data/lyx/hubs/SpaTD/stdgcn/benchmark/MERFISH/ID1/Bregma0.01/stride_predict_result.csv")
spot_celltype_array_norm_df.to_csv(os.path.join(out_dir, "stride_predict_result_seed{}.csv".format(seed)))

In [88]:
pred_proportion= pd.read_csv(os.path.join(out_dir, "stride_predict_result_seed{}.csv".format(seed)),index_col=0)
pred_proportion.columns = ['Astrocyte', 'Endothelial', 'Excitatory_neuron', 'Interneuron',
       'Microglia', 'Oligodendrocyte']
#pd.read_csv('./benchmark/MERFISH/ID1/Bregma0.01/SONAR.results.csv',index_col=0)
import pandas as pd
columns = pd.Index(pred_proportion.columns, dtype='object')
columns

Index(['Astrocyte', 'Endothelial', 'Excitatory_neuron', 'Interneuron',
       'Microglia', 'Oligodendrocyte'],
      dtype='object')

In [89]:
# real_proportion = pd.read_csv('./benchmark_data/MERFISH/ST_data/MERFISH_ID1_ground_truth_0.01.tsv', sep='\t')
real_proportion = pd.read_table(os.path.join(data_dir,'ST_data',"seqFishplus_Cortex_200_ground_truth_all.tsv"),index_col=0)
#real_proportion = pd.DataFrame(data=real_proportion_raw.iloc[:, 4:].values,columns=columns,index=real_proportion_raw.iloc[:, 0])
real_proportion = real_proportion[columns]
real_proportion.head()

,Astrocyte,Endothelial,Excitatory_neuron,Interneuron,Microglia,Oligodendrocyte
gr_27,0.071429,0.071429,0.571429,0.071429,0.071429,0.142857
gr_24,0.000000,0.000000,0.700000,0.100000,0.200000,0.000000
gr_26,0.000000,0.000000,0.777778,0.000000,0.000000,0.222222
gr_36,0.000000,0.125000,0.750000,0.125000,0.000000,0.000000
gr_35,0.000000,0.111111,0.777778,0.000000,0.000000,0.111111


In [90]:
rmse_by_K = 0
rmse_list = []
for column in columns:
    _sum = np.sum(np.square(pred_proportion[column]-real_proportion[column]))
    rmse_by_K += _sum
    rmse_list.append(_sum)
rmse_by_K /= len(columns)
rmse_by_K = np.sqrt(rmse_by_K)
rmse_by_K 

1.1370986515816737

In [91]:
JSD = JS_divergence_mat(pred_proportion, real_proportion)/pred_proportion.shape[0]
JSD

0.15700883411234634

In [92]:
pcc_by_K = 0
pcc_list=[]
for column in columns:
    _sum = np.corrcoef(pred_proportion[column],real_proportion[column])[0][1]
    pcc_by_K +=_sum
    pcc_list.append(_sum)
pcc_by_K /=len(columns)
pcc_by_K

0.45495260867018983

In [93]:
pd.DataFrame(zip(columns,rmse_list,pcc_list),columns=['cell_type','RMSE','PCC']).to_csv(os.path.join(out_dir,'stride_celltype_predict_result_seed{}.csv').format(seed))

In [94]:
with open('/data/lyx/hubs/SpaTD/stdgcn/benchmark/final_stat.txt','a+') as df:
    #df.write('data\tseed\tRMSE\tPCC\tJSD\n')
    df.write('{}\t{}\t{}\t{}\t{}\n'.format(data_dir.split('/')[-1],seed,rmse_by_K,pcc_by_K,JSD))
df.close()

In [95]:
out_dir

'/data/lyx/hubs/SpaTD/stdgcn/benchmark/seqFISH_plus/Dataset2_seqFISHplus_AllenVIsp/'

In [ ]:
cd